In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Ranking of feature importance for genes.

In [2]:
allImp = np.load("NormalizedImp100.npy")
gene = np.load('../../data/geneAfterDiscard_0.npy', allow_pickle=True)

impSum = allImp.sum(axis = 0)

geneIdx = [i for i in range(gene.shape[0])]

toSort = zip(geneIdx, impSum)
sortedImp = sorted(toSort, key=lambda x: x[1], reverse=1)

sortedImp = np.array(list(zip(*sortedImp)))

sortedIdx = sortedImp[0].astype(int)
print(sortedIdx.shape)
print(sortedIdx)

(11959,)
[ 3774  1667  2429 ... 11944 11947 11951]


In [3]:
data = np.load('../../data/oversampled_data.npy', allow_pickle=True)

feature = data[:, :-1]
label = data[:, -1]
print(gene.shape)
print(feature.shape)
print(label.shape)

(11959,)
(296, 11959)
(296,)


# Select the satisfactory gene.

In [4]:
ISGnum = 60

ISGidx = sortedIdx[:ISGnum]

ISGfeature = feature[:, ISGidx]
ISG = gene[ISGidx]

# calculate GIC

In [5]:
TGidx = np.where(label > 0.5)
SGidx = np.where(label < 0.5)

TGISGfeature = ISGfeature[TGidx]
TGlabel = label[TGidx]
SGISGfeature = ISGfeature[SGidx]
SGlabel = label[SGidx]

In [6]:
mean = SGISGfeature.mean(axis = 0)
std = SGISGfeature.std(axis = 0)

In [7]:
from scipy.special import comb, perm

def calculateGIC(x, mean, std, upper_bar = 0.2, lower_bar = -0.2):
    
    sampleNum = x.shape[0]
    ISGNUM = x.shape[1]
    
    b = (x - mean) / std

    c = np.zeros(b.shape, dtype = 'int')
    c = np.where(b > upper_bar, 1, c)
    c = np.where(b < lower_bar, -1, c)
    
    pariNum = int(comb(ISGNUM, 2))
    d = np.zeros((sampleNum, pariNum), dtype = 'float')
    cnt = 0
    for i in range(ISGNUM):
        for j in range(i + 1, ISGNUM):
            d[:, cnt] = c[:, i] * c[:, j]
            cnt += 1
    
    return d

In [8]:
upper_bar = 0.2
lower_bar = -0.2
genePairName = []
for i in range(ISGnum):
    for j in range(i + 1, ISGnum):
        genePairName.append([ISG[i], ISG[j]])

genePairName = np.array(genePairName)

TG_d = calculateGIC(TGISGfeature, mean, std, upper_bar, lower_bar)
SG_d = calculateGIC(SGISGfeature, mean, std, upper_bar, lower_bar)

X_train, X_test, y_train, y_test = train_test_split(ISGfeature, label, test_size=0.3, shuffle=True, random_state=2022)
X_train_d = calculateGIC(X_train, mean, std, upper_bar, lower_bar)
X_test_d = calculateGIC(X_test, mean, std, upper_bar, lower_bar)


In [48]:
# signature gene pair

threshold = 0.645
retainIdx = abs(TG_d.sum(axis = 0)) >= threshold * TG_d.shape[0]
SGP_X_train = X_train_d[:, retainIdx]
SGP_X_test = X_test_d[:, retainIdx]

selectedGenePair = genePairName[retainIdx]
print(SGP_X_train.shape)
print(selectedGenePair.shape)
used_ISG_gene_num = len(set(selectedGenePair.reshape(-1)))
print(f'use {used_ISG_gene_num} genes')

(207, 8)
(8, 2)
使用了9个基因


# SVM test for signature gene pairs

In [49]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.svm import LinearSVC
from sklearn import svm

In [51]:
acc = []
prec = []
rec = []
auc = []

numExperiments = 5

for i in range(numExperiments):
    lsvc = svm.SVC(probability=True) 
    lsvc.fit(SGP_X_train,y_train)                

    y_pred = lsvc.predict(SGP_X_test)
    acc.append(accuracy_score(y_test, y_pred))
    prec.append(precision_score(y_test, y_pred))
    rec.append(recall_score(y_test, y_pred))
    predict_prob_y = lsvc.predict_proba(SGP_X_test)[:, 1]
    auc.append(roc_auc_score(y_test, predict_prob_y))

acc = np.array(acc)
prec = np.array(prec)
rec = np.array(rec)
auc = np.array(auc)

print('{}\t{}\t{}\t{}\t{}'.format(used_ISG_gene_num, acc.mean(), prec.mean(), rec.mean(),  auc.mean()))

9	0.7865168539325843	0.8	0.7441860465116279	0.8844792719919109
